In [1]:
import mysql.connector
import pandas as pd

In [2]:
mydb=mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='***********',
    database='sakila'
)

In [3]:
mycursor=mydb.cursor()
# crate acess to database
query='show tables;'
# query
mycursor.execute(query)
# execute query
result=mycursor.fetchall()
# save result
df=pd.DataFrame(result,columns=mycursor.column_names)
df.head()

,Tables_in_sakila
0,actor
1,actor_info
2,address
3,category
4,city


In [5]:
mycursor=mydb.cursor()
def sql(query):
    mycursor.execute(query)
    result=mycursor.fetchall()
    df=pd.DataFrame(result,columns=mycursor.column_names)
    return df

In [8]:
# Gunakan hanya base table
a = sql('''SHOW FULL TABLES;''')
a.loc[a["Table_type"]=="BASE TABLE"]

,Tables_in_sakila,Table_type
0,actor,BASE TABLE
2,address,BASE TABLE
3,category,BASE TABLE
4,city,BASE TABLE
5,country,BASE TABLE
6,customer,BASE TABLE
8,film,BASE TABLE
9,film_actor,BASE TABLE
10,film_category,BASE TABLE
12,film_text,BASE TABLE


## 1. 10 film komedi dengan durasi tersingkat

In [28]:
sql('''
SELECT film.title as Title, category.name as Category, film.length as Length
FROM film, category, film_category WHERE
category.category_id = film_category.category_id AND
film_category.film_id = film.film_id AND
category.name = "Comedy"
ORDER BY Length
LIMIT 10;
''')

,Title,Category,Length
0,DOWNHILL ENOUGH,Comedy,47
1,PARADISE SABRINA,Comedy,48
2,HEAVEN FREEDOM,Comedy,48
3,HURRICANE AFFAIR,Comedy,49
4,LION UNCUT,Comedy,50
5,ZORRO ARK,Comedy,50
6,CLOSER BANG,Comedy,58
7,AIRPLANE SIERRA,Comedy,62
8,LONELY ELEPHANT,Comedy,67
9,DOOM DANCING,Comedy,68


## 2. kategori film beserta jumlah film tiap kategori & rata-rata harga sewa DVD film tiap kategori

In [35]:
sql('''
SELECT category.name as Category, COUNT(film.title) as Jumlah_Movie, AVG(film.rental_rate) as Rata_Harga_Sewa
FROM film, category, film_category WHERE
category.category_id = film_category.category_id AND
film_category.film_id = film.film_id
GROUP BY Category
ORDER BY Jumlah_movie DESC
''')

,Category,Jumlah_Movie,Rata_Harga_Sewa
0,Sports,74,3.125135
1,Foreign,73,3.099589
2,Family,69,2.758116
3,Documentary,68,2.666471
4,Animation,66,2.808182
5,Action,64,2.646250
6,New,63,3.116984
7,Drama,62,3.022258
8,Games,61,3.252295
9,Sci-Fi,61,3.219508


,kategori,jumlahMovie,rataHargaSewa
0,Sports,74,3.125135
1,Foreign,73,3.099589
2,Family,69,2.758116
3,Documentary,68,2.666471
4,Animation,66,2.808182
5,Action,64,2.646250
6,New,63,3.116984
7,Drama,62,3.022258
8,Games,61,3.252295
9,Sci-Fi,61,3.219508


## 3. 10 aktor/aktris yang paling banyak membintangi film

In [46]:
sql('''
SELECT actor.actor_id as Actor_ID, actor.first_name as First_Name, actor.last_name as Last_Name, COUNT(film_actor.film_id) as Jumlah_Movie
FROM actor, film_actor
WHERE actor.actor_id = film_actor.actor_id
GROUP BY Actor_ID
ORDER BY Jumlah_Movie DESC
LIMIT 10;
''')

,Actor_ID,First_Name,Last_Name,Jumlah_Movie
0,107,GINA,DEGENERES,42
1,102,WALTER,TORN,41
2,198,MARY,KEITEL,40
3,181,MATTHEW,CARREY,39
4,23,SANDRA,KILMER,37
5,81,SCARLETT,DAMON,36
6,37,VAL,BOLGER,35
7,158,VIVIEN,BASINGER,35
8,106,GROUCHO,DUNST,35
9,13,UMA,WOOD,35


,actor_id,first_name,last_name,jumlah_Movie
0,107,GINA,DEGENERES,42
1,102,WALTER,TORN,41
2,198,MARY,KEITEL,40
3,181,MATTHEW,CARREY,39
4,23,SANDRA,KILMER,37
5,81,SCARLETT,DAMON,36
6,158,VIVIEN,BASINGER,35
7,144,ANGELA,WITHERSPOON,35
8,106,GROUCHO,DUNST,35
9,60,HENRY,BERRY,35


## 4. kategori film beserta jumlah film yang pernah dibintangi oleh Gina Degeneres

,category,jumlah_Movie
0,Documentary,3
1,Foreign,2
2,Music,4
3,New,1
4,Sci-Fi,7
5,Action,3
6,Drama,2
7,Animation,4
8,Horror,1
9,Family,4


In [52]:
sql('''
SELECT category.name as Category, COUNT(film_actor.film_id) as Jumlah_Movie
FROM category, film_category, film_actor, actor
WHERE
category.category_id = film_category.category_id AND
film_actor.film_id = film_category.film_id AND
film_actor.actor_id = actor.actor_id AND
actor.first_name = "Gina" AND actor.last_name = "Degeneres"
GROUP BY Category
''')

,Category,Jumlah_Movie
0,Documentary,3
1,Foreign,2
2,Music,4
3,New,1
4,Sci-Fi,7
5,Action,3
6,Drama,2
7,Animation,4
8,Horror,1
9,Family,4


## 5. judul film sci-fi yang pernah dibintangi oleh Gina Degeneres

,title,category
0,CHARIOTS CONSPIRACY,Sci-Fi
1,COLDBLOODED DARLING,Sci-Fi
2,FRISCO FORREST,Sci-Fi
3,GOODFELLAS SALUTE,Sci-Fi
4,LICENSE WEEKEND,Sci-Fi
5,OPEN AFRICAN,Sci-Fi
6,SPIRITED CASUALTIES,Sci-Fi


In [59]:
sql('''
SELECT film.title as Title, category.name as Category
FROM category, film_category, film_actor, actor, film
WHERE
film_category.film_id = film.film_id AND
category.category_id = film_category.category_id AND
film_actor.film_id = film_category.film_id AND
film_actor.actor_id = actor.actor_id AND
actor.first_name = "Gina" AND actor.last_name = "Degeneres" AND
category.name="Sci-Fi"
''')

,Title,Category
0,CHARIOTS CONSPIRACY,Sci-Fi
1,COLDBLOODED DARLING,Sci-Fi
2,FRISCO FORREST,Sci-Fi
3,GOODFELLAS SALUTE,Sci-Fi
4,LICENSE WEEKEND,Sci-Fi
5,OPEN AFRICAN,Sci-Fi
6,SPIRITED CASUALTIES,Sci-Fi
